In [1]:
# 한국어 형태소 분석기
## kiwipiepy.Kiwi() 사용
## .tokenize() 함수를 이용해서 분석
## 분석 결과는 해당 형태소(.form), 형태소 품사(.tag), 형태소 위치(.start), 형태소 길이(.len) 를 같이 출력
## 형태소만 따로 골라내려면 for문을 이용해서 별개로 추출해야 함

import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
import numpy
import pandas
from IPython.display import display

# from konlpy.tag import Kkma
from kiwipiepy import Kiwi #클래스 형태로 구현됨
from tqdm import tqdm
# import threading

In [2]:
# 사용 방법
kiwi = Kiwi() # 클래스를 객체로 생성하는데 시간이 꽤 걸려서 객체를 생성하고 시작해야함

text = "이 영화는 재미있습니다. 추천합니다."
results = kiwi.tokenize(text)
print(results)

#단어만 추출
words = [] #단어 저장 리스트 따로 선언
for i in range(len(results)) :
    words.append(results[i].form) #for 문 내에서 하나씩 집어넣어야 함

print(words)
print(" ".join(words))

[Token(form='이', tag='MM', start=0, len=1), Token(form='영화', tag='NNG', start=2, len=2), Token(form='는', tag='JX', start=4, len=1), Token(form='재미있', tag='VA', start=6, len=3), Token(form='습니다', tag='EF', start=9, len=3), Token(form='.', tag='SF', start=12, len=1), Token(form='추천', tag='NNG', start=14, len=2), Token(form='하', tag='XSV', start=16, len=1), Token(form='ᆸ니다', tag='EF', start=16, len=3), Token(form='.', tag='SF', start=19, len=1)]
['이', '영화', '는', '재미있', '습니다', '.', '추천', '하', 'ᆸ니다', '.']
이 영화 는 재미있 습니다 . 추천 하 ᆸ니다 .


In [3]:
# 구동하다가 분석하는데 10초 이상 걸리는 문장을 넘기는 서브스레드를 가진 분석 class를 만들어 보도록 합시다
# kiwi는 속도가 빨라서 필요가 없습니다.

# class Preprocessor() :
#     def __init__(self) :
#         self._result = None #스레드 결과값 저장 변수
#         self._error = None #스레드 오류 발생시 오류 저장 변수
#     def _process_with_timeout(self, s : str, timeout : int) :
#         def work() :
#             try :
#                 self._result = Kkma().morphs(s)
#             except Exception as e:
#                 self._error = e
#         self._error = None
#         thread = threading.Thread(target = work, daemon = True) #work 함수 구동할 스레드 선언
#         thread.start() #해당 스레드 동작 시작
#         thread.join(timeout = timeout) #해당 스레드 timeout초동안 기다려 주고

#         if thread.is_alive() : #기다렸는데도 아직 동작이 안끝났으면
#             raise Exception(f"Error : {timeout}초 시간 초과")
#         if self._error is not None :
#             raise Exception(str(self._error))

#         return self._result
#     def process(self, s : str, timeout : int = 10) :
#         result = self._process_with_timeout(s, timeout)
#         return result

In [4]:
## 본격적으로 데이터 전처리 시작, 데이터 불러오기
original_df = pandas.read_csv("data/ratings_test.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [5]:
df = original_df[['document', 'label']]
kiwi = Kiwi() #객체 생성

for i in tqdm(range(len(df))) :
    text = str(df.iloc[i,0]) #문장 하나씩 가져와서
    text = custom.text_preprocess_kor(text, end_mark=True, chosung=False) #문장의 특문 제거, ?!는 놔두고, 초성은 제거
    if text == " " : #예외 : 전처리 했더니 공백이 된 문자
        df.iloc[i,0] = text 
        continue
    try :
        words = [] #단어 저장 리스트 따로 선언
        temp = kiwi.tokenize(text) #단어 형태소 분석결과를 temp에 저장
        for j in range(len(temp)) :
            words.append(temp[j].form) #단어 부분만 선택해서 words에 저장
        df.iloc[i,0] = " ".join(words) #list를 하나의 문장으로 묶어서 저장
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}") #어떤 문장에서 오류가 생겼는지 확인
        
display(df)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:00<00:00, 827.94it/s]


,document,label
0,굳,1
1,,0
2,뭐 이 야 이 평점 들 은 . 나쁘 지 ᆫ 않 지만 점 짜리 는 더더욱 아니 잖아,0
3,지루하 지 는 않 은데 완전 막장 이 ᆷ . 돈 주 고 보 기 에 는 .,0
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ᆫ데 . 왜 로 나오 어서 저 의 ...,0
...,...,...
49995,오랜만 에 평점 로 이 기 ᆫ 하 었 네 킹왕짱 쌈 뽕 하 ᆫ 영화 를 만나 었 습...,1
49996,의지 박약 들 이나 하 는 거 다 탈영 은 일단 주인공 김대희 닮 었 고 이등병 찐따,0
49997,그림 도 좋 고 완성도 도 높 었 지만 . 보 는 내내 불안 하 게 만들 ᆫ다,0
49998,절대 보 어서 는 안 되 ᆯ 영화 . 재미 도 없 고 기분 만 잡치 고 . 한 세트...,0


In [6]:
word_len = []
for i in tqdm(range(len(df))) :
    len_ = len(df.iloc[i,0].split())
    word_len.append(len_)
df["word_len"] = word_len

df = df[df["word_len"] > 0]

display(df)

100%|█████████████████████████████████████████████████████████████████████████| 50000/50000 [00:01<00:00, 35815.69it/s]


,document,label,word_len
0,굳,1,1
2,뭐 이 야 이 평점 들 은 . 나쁘 지 ᆫ 않 지만 점 짜리 는 더더욱 아니 잖아,0,19
3,지루하 지 는 않 은데 완전 막장 이 ᆷ . 돈 주 고 보 기 에 는 .,0,18
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ᆫ데 . 왜 로 나오 어서 저 의 ...,0,28
5,음악 이 주 가 되 ᆫ 최고 의 음악 영화,1,10
...,...,...,...
49995,오랜만 에 평점 로 이 기 ᆫ 하 었 네 킹왕짱 쌈 뽕 하 ᆫ 영화 를 만나 었 습...,1,25
49996,의지 박약 들 이나 하 는 거 다 탈영 은 일단 주인공 김대희 닮 었 고 이등병 찐따,0,18
49997,그림 도 좋 고 완성도 도 높 었 지만 . 보 는 내내 불안 하 게 만들 ᆫ다,0,18
49998,절대 보 어서 는 안 되 ᆯ 영화 . 재미 도 없 고 기분 만 잡치 고 . 한 세트...,0,26


In [7]:
df.to_csv("data/korean_movie_test.txt", sep = "\t", encoding="UTF8")